# Get Imports

In [2]:
from functions import getClasses
from functions import createFolder
import csv
import os
import matplotlib.pyplot as plt
import sys
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm

ourClasses, _, _, _ = getClasses('classes.csv', "data/ontology.json", 'data/class_labels_indices.csv')

# Get Generated Features

In [7]:
#Assumes featureFolder is generated in CSV
def getFeatureMatrices(pathToFeatureCSV):
    data = [];
    names = [];
    numComma = pathToFeatureCSV.count(',')
    with open(pathToFeatureCSV, 'r') as f:
        csvReader = csv.reader(f, delimiter=',')
        rowCount = 0
        for row in csvReader:
            rowCount = rowCount + 1
            
            names.append(row[0][row[0].rfind("/")+1:])
            data.append(np.array(row[1+numComma:]).astype(np.float))
                            
    return np.array(data), names

# Get an equal positive set and negative set for a class

In [4]:
def getFeatureSetPosNeg(pathToDirWithCSV, cName, ourClasses):
    data_pos, names = getFeatureMatrices(pathToDirWithCSV + cName + ".csv")
    amountOfNegData = data_pos.shape[0]
    
    data_neg = [];
    
    for c in ourClasses:
        if c == cName:
            continue
        data_new, names_new = getFeatureMatrices(pathToDirWithCSV + c + ".csv")
        negDataCounter = 0
        for i in range(0, len(names_new)):
            
            if names_new[i] in names:
                continue
                
            negDataCounter = negDataCounter + 1
            data_neg.append(data_new[i,:])
            
            if negDataCounter >= amountOfNegData/6:
                break

    return data_pos, np.array(data_neg)

# Calculates test error given a list of classifiers

In [5]:
def getTestErrorValue(clf_list, ourClasses, pathToDirWithTestCSV):
    numCorrect_test  = 0
    num_test  = 0
    
    dataSets = [];
    nameSets = [];
    for c in ourClasses:
        data, names = getFeatureMatrices(pathToDirWithTestCSV + c + ".csv")
        dataSets.append(data)
        nameSets.append(names)
    
    for k in range(0, len(ourClasses)):
        data = dataSets[k]
        names = nameSets[k]
        
        for i in range(0, len(clf_list)):
            clf = clf_list[i]
            result = clf.predict(data)
            
            for j in range(0, len(names)):
                num_test = num_test + 1
                if result[j]  and names[j] in nameSets[i]:
                    numCorrect_test = numCorrect_test + 1
                    
                if not(result[j]) and not(names[j] in nameSets[i]):
                    numCorrect_test = numCorrect_test + 1

    return 1 - float(numCorrect_test)/float(num_test)

# Performing LDA

In [6]:
print("Using LDA")

clf_list = []

for c in ourClasses:
    data_pos, data_neg = getFeatureSetPosNeg("data/CNN features/unbal_tr_feat/", c, ourClasses)
    dataset = np.vstack([data_pos, data_neg])
    labels  = np.ravel(np.vstack([np.ones([data_pos.shape[0],1]), np.zeros([data_neg.shape[0],1])]))

    clf = LDA()
    clf.fit(dataset, labels);
    LDA(n_components=None, priors=None, shrinkage=None, solver='svd',store_covariance=False, tol=0.1)
    print("Training classifier " + c + " training error: {}".format(1 - clf.score(dataset,labels)))
    clf_list.append(clf)

testE = getTestErrorValue(clf_list, ourClasses, "data/CNN features/eval_feat/")
print("Test Error  : {}".format(testE))

Using LDA


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [10]:
print(data_pos.shape)
print(data_neg.shape)

(35042, 1280)
(35040,)
